# analyze_images.py 코드 리뷰 노트북

각 셀을 하나씩 실행하면서 변수가 뭔지 확인하자.

**실행 방법**: 셀 클릭 → `Shift+Enter`

## 0. 준비

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# 한글 폰트
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 이미지 보여주는 헬퍼 함수
def show(title, img, cmap=None):
    plt.figure(figsize=(4, 6))
    if cmap:
        plt.imshow(img, cmap=cmap)
    else:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

print('준비 완료!')

## 1. 이미지 로드

OG = 부품 전체가 보이는 원본 이미지  
Cropped = 부품 하나를 잘라낸 ROI 이미지

In [ ]:
# 경로 설정
DATA_DIR = Path('../data/imgae_processing')
PART_NAME = '922101-08080730'

og_dir = DATA_DIR / 'OG' / PART_NAME
crop_dir = DATA_DIR / 'Cropped' / PART_NAME

# OG 이미지 1장 로드
og_path = sorted(og_dir.glob('*.png'))[0]
og_image = cv2.imread(str(og_path))

print(f'OG 이미지: {og_path.name}')
print(f'크기: {og_image.shape}  (세로, 가로, 채널)')

show('OG 이미지 (부품 전체)', og_image)

In [ ]:
# Cropped 이미지 1장 로드
crop_path = sorted(crop_dir.glob('*.png'))[0]
crop_image = cv2.imread(str(crop_path))

print(f'Cropped 이미지: {crop_path.name}')
print(f'크기: {crop_image.shape}  (세로, 가로, 채널)')

show('Cropped 이미지 (ROI)', crop_image)

---
## 2. extract_board_color_from_og() — 기판색 추출

OG 이미지에서 녹색 기판 영역을 찾아 Lab 기준색을 구한다.

In [ ]:
# Step 2-1: BGR → HSV 변환
hsv = cv2.cvtColor(og_image, cv2.COLOR_BGR2HSV)

print(f'hsv.shape = {hsv.shape}')
print(f'한 픽셀 예시: H={hsv[0,0,0]}, S={hsv[0,0,1]}, V={hsv[0,0,2]}')

In [ ]:
# Step 2-2: 녹색 범위만 추출 (Hue 35~95)
board_mask = cv2.inRange(
    hsv,
    np.array([35, 30, 20]),   # 최소: H=35, S=30, V=20
    np.array([95, 255, 255])  # 최대: H=95, S=255, V=255
)

print(f'board_mask.shape = {board_mask.shape}')
print(f'흰색(기판) 픽셀 수: {np.sum(board_mask == 255)}')
print(f'검정(기판 아님) 픽셀 수: {np.sum(board_mask == 0)}')

show('기판 마스크 (흰색=기판)', board_mask, cmap='gray')

In [ ]:
# Step 2-3: 노이즈 제거 (MORPH_OPEN)
kernel = np.ones((3, 3), np.uint8)
board_mask = cv2.morphologyEx(board_mask, cv2.MORPH_OPEN, kernel)

print(f'노이즈 제거 후 흰색 픽셀: {np.sum(board_mask == 255)}')

# 원본에 기판 영역 초록색으로 오버레이
overlay = cv2.cvtColor(og_image, cv2.COLOR_BGR2RGB).copy()
overlay[board_mask > 0] = [0, 255, 0]
show('기판 영역 오버레이 (초록)', overlay)

In [ ]:
# Step 2-4: 기판 영역의 Lab 중앙값 = 기준색
lab = cv2.cvtColor(og_image, cv2.COLOR_BGR2Lab)
board_pixels = lab[board_mask > 0].astype(np.float64)

print(f'기판 픽셀 수: {len(board_pixels)}')
print(f'기판 픽셀 예시 (첫 5개):')
for p in board_pixels[:5]:
    print(f'  L={p[0]:.1f}, a={p[1]:.1f}, b={p[2]:.1f}')

board_ref_lab = np.median(board_pixels, axis=0)
print(f'\n>>> 기판 기준색 (Lab): L={board_ref_lab[0]:.1f}, a={board_ref_lab[1]:.1f}, b={board_ref_lab[2]:.1f}')

---
## 3. measure_fillet() — Stage 1: 전처리

In [ ]:
# Stage 1: 바이래터럴 필터 (에지 보존 노이즈 제거)
denoised = cv2.bilateralFilter(crop_image, d=5, sigmaColor=50, sigmaSpace=50)

fig, axes = plt.subplots(1, 2, figsize=(8, 6))
axes[0].imshow(cv2.cvtColor(crop_image, cv2.COLOR_BGR2RGB))
axes[0].set_title('원본')
axes[0].axis('off')
axes[1].imshow(cv2.cvtColor(denoised, cv2.COLOR_BGR2RGB))
axes[1].set_title('노이즈 제거 후')
axes[1].axis('off')
plt.tight_layout()
plt.show()

---
## 4. measure_fillet() — Stage 2: 기판 거리맵 (시각화용)

In [ ]:
# Line 101: BGR → Lab
lab_crop = cv2.cvtColor(denoised, cv2.COLOR_BGR2Lab).astype(np.float64)
print(f'lab_crop.shape = {lab_crop.shape}')
print(f'한 픽셀 예시: L={lab_crop[0,0,0]:.1f}, a={lab_crop[0,0,1]:.1f}, b={lab_crop[0,0,2]:.1f}')

In [ ]:
# Line 102-103: 기판색과의 거리 계산
diff = lab_crop - board_ref_lab
distance = np.sqrt(np.sum(diff ** 2, axis=2)).astype(np.float32)

print(f'distance.shape = {distance.shape}')
print(f'최소 거리: {distance.min():.1f} (기판과 같은 색)')
print(f'최대 거리: {distance.max():.1f} (기판과 가장 다른 색)')

In [ ]:
# Line 104: 0~255로 정규화 (시각화용)
dist_norm = cv2.normalize(distance, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

show('기판과의 거리맵 (밝을수록 다름)', dist_norm, cmap='hot')

---
## 5. measure_fillet() — Stage 3: K-means 클러스터링

In [ ]:
img_h, img_w = crop_image.shape[:2]
kmeans_k = 3  # 3그룹으로 나누기

# Lab 변환 → 1줄로 펼치기
lab_f32 = cv2.cvtColor(denoised, cv2.COLOR_BGR2Lab).astype(np.float32)
pixels_lab = lab_f32.reshape(-1, 3)

print(f'이미지 크기: {img_w}x{img_h} = {img_w * img_h} 픽셀')
print(f'pixels_lab.shape = {pixels_lab.shape}  (픽셀수, Lab 3채널)')

# 위치 정보 추가
pos_weight = 10
ys, xs = np.mgrid[0:img_h, 0:img_w]
xs_norm = (xs.reshape(-1, 1).astype(np.float32) / max(img_w, 1)) * pos_weight
ys_norm = (ys.reshape(-1, 1).astype(np.float32) / max(img_h, 1)) * pos_weight
features = np.hstack([pixels_lab, xs_norm, ys_norm])

print(f'features.shape = {features.shape}  (픽셀수, [L,a,b,x,y])')
print(f'첫 픽셀: L={features[0,0]:.1f}, a={features[0,1]:.1f}, b={features[0,2]:.1f}, x={features[0,3]:.1f}, y={features[0,4]:.1f}')

In [ ]:
# K-means 실행
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 0.5)
_, labels, _ = cv2.kmeans(
    features, kmeans_k, None, criteria, 5, cv2.KMEANS_PP_CENTERS
)
labels = labels.flatten()

print(f'labels.shape = {labels.shape}')
print(f'그룹 종류: {np.unique(labels)}')
for k in range(kmeans_k):
    count = np.sum(labels == k)
    print(f'  그룹 {k}: {count}px ({count/(img_w*img_h)*100:.1f}%)')

# 클러스터 시각화
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255)]
kmeans_vis = np.zeros((img_h, img_w, 3), dtype=np.uint8)
for k in range(kmeans_k):
    kmeans_vis[labels.reshape(img_h, img_w) == k] = colors[k % len(colors)]

show('K-means 결과 (색=그룹)', kmeans_vis)

---
## 6. measure_fillet() — Stage 4: 솔더 그룹 선택 (B/R 비율)

In [ ]:
# 각 그룹의 평균 BGR 색상과 B/R 비율
bgr_flat = denoised.reshape(-1, 3).astype(np.float32)
fillet_clusters = []

print('각 그룹의 평균 색상:')
print(f'{"그룹":>4} | {"B":>6} | {"G":>6} | {"R":>6} | {"B/R":>6} | 판정')
print('-' * 55)

for k in range(kmeans_k):
    cluster_mask = labels == k
    if np.sum(cluster_mask) < 30:
        continue
    mean_bgr = np.mean(bgr_flat[cluster_mask], axis=0)
    b_mean, g_mean, r_mean = mean_bgr
    ratio = b_mean / max(r_mean, 1)
    
    is_fillet = ratio > 0.8
    if is_fillet:
        fillet_clusters.append(k)
    
    mark = '◀ 솔더!' if is_fillet else ''
    print(f'{k:>4} | {b_mean:>6.1f} | {g_mean:>6.1f} | {r_mean:>6.1f} | {ratio:>6.2f} | {mark}')

print(f'\n>>> 솔더 그룹: {fillet_clusters}')

In [ ]:
# 솔더 마스크 생성
fillet_mask = np.zeros((img_h, img_w), dtype=np.uint8)
for k in fillet_clusters:
    fillet_mask[labels.reshape(img_h, img_w) == k] = 255

print(f'솔더 픽셀: {np.sum(fillet_mask == 255)}px')

show('솔더 마스크 (B/R > 0.8)', fillet_mask, cmap='gray')

---
## 7. measure_fillet() — Stage 5: 후처리 + 면적

In [ ]:
# 모폴로지 (OPEN만 - CLOSE는 역효과 가능)
kernel = np.ones((3, 3), np.uint8)
mask_opened = cv2.morphologyEx(fillet_mask, cv2.MORPH_OPEN, kernel)
mask_closed = cv2.morphologyEx(mask_opened, cv2.MORPH_CLOSE, kernel)

print(f'모폴로지 전: {np.sum(fillet_mask == 255)}px')
print(f'OPEN 후:    {np.sum(mask_opened == 255)}px')
print(f'CLOSE 후:   {np.sum(mask_closed == 255)}px')

fig, axes = plt.subplots(1, 3, figsize=(12, 6))
axes[0].imshow(fillet_mask, cmap='gray')
axes[0].set_title('모폴로지 전')
axes[1].imshow(mask_opened, cmap='gray')
axes[1].set_title('OPEN 후')
axes[2].imshow(mask_closed, cmap='gray')
axes[2].set_title('OPEN+CLOSE 후')
for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# 면적 계산
PIXEL_SIZE_MM = 0.00465
clean_mask = mask_closed  # 현재 코드는 OPEN+CLOSE 사용

contours, _ = cv2.findContours(clean_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(f'찾은 덩어리 수: {len(contours)}')

area_pixels = 0
min_area = max(5, clean_mask.size * 0.005)
print(f'최소 면적 기준: {min_area:.0f}px')

for i, c in enumerate(contours):
    a = cv2.contourArea(c)
    keep = '✅' if a >= min_area else '❌ (너무 작음)'
    print(f'  덩어리 {i}: {int(a)}px {keep}')
    if a >= min_area:
        area_pixels += int(a)

area_mm2 = area_pixels * (PIXEL_SIZE_MM ** 2)
print(f'\n>>> 최종 면적: {area_pixels}px = {area_mm2:.6f} mm²')

---
## 8. 최종 결과 한눈에 보기

In [ ]:
# 전체 파이프라인 결과 비교
overlay = cv2.cvtColor(crop_image, cv2.COLOR_BGR2RGB).copy()
overlay[clean_mask > 0] = [255, 0, 0]  # 솔더 = 빨강

fig, axes = plt.subplots(1, 5, figsize=(18, 6))

axes[0].imshow(cv2.cvtColor(crop_image, cv2.COLOR_BGR2RGB))
axes[0].set_title('1. 원본')

axes[1].imshow(dist_norm, cmap='hot')
axes[1].set_title('2. 거리맵')

axes[2].imshow(cv2.cvtColor(kmeans_vis, cv2.COLOR_BGR2RGB))
axes[2].set_title('3. K-means')

axes[3].imshow(clean_mask, cmap='gray')
axes[3].set_title(f'4. 솔더 마스크\n{area_pixels}px')

axes[4].imshow(overlay)
axes[4].set_title('5. 오버레이')

for ax in axes:
    ax.axis('off')

plt.suptitle(f'파이프라인 전체 흐름 — {area_mm2:.6f} mm²', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()